In [ ]:
import pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession


def get_spark_context(app_name: str):
    conf = SparkConf()
    
    conf.setAll(
        [
            ('spark.master', 'spark://spark-master:7077'),
            ('spark.driver.host', 'local[*]'),
            ('spark.submit.deployMode', 'client'),
            ('spark.driver.bindAddress', '192.168.0.15'),
            ('spark.app.name', app_name),
        ]
    )
    
    return SparkSession.builder.config(conf=conf).getOrCreate()

In [ ]:
spark = get_spark_context('netology')

выше возникает какая-то беда при подключении к docker-образу спарка, много вариантов перепробовал, порты 8080, 7070 открыты, разные настройки адреса передавал, все время большой лог ошибок и в конце:
WARN StandaloneAppClient$ClientEndpoint: Drop UnregisterApplication(null) because has not yet connected to master

на одном из форумов в итоге вычитал, что возможно проблема в разных версиях java на машине и в контейнере, но сил моральных уже не осталось

через консоль прекрасно подключается к контейнеру

Само решение на pyspark ниже:

In [ ]:
data = sc.TextFile('movies.csv')

In [ ]:
stop_words = ['The', 'In', 'I', 'Mr.', 'a']

In [ ]:
(
 data.map(lambda x: (x.split(',')[1].split(' ')[0], 1))
     .filter(lambda x: x[0] not in stop_words)
     .reduceByKey(lambda a, b: a + b) \
     .takeOrdered(15, lambda x: -x[1])
)